# Homework 5

> **NOTE:** Beyler repoya ekledigim .bat dosyası ile git filter eklemis oluyorsunuz, böylece gereksiz degisiklikleri commit etmeyerek conflict ihtimalimiz cok daha düsüyor. Herkesin yapmasını tavsiye ediyorum.

In [ ]:
import pandas as pd
import numpy as np
from sympy import Symbol, lambdify
# import random

In [ ]:
x1 = Symbol("x1")
x2 = Symbol("x2")
x3 = Symbol("x3")
x4 = Symbol("x4")

func = 3*x1**2 + 2*x2**2 - 2*x1*x2 - 4*x1 + 2*x2 + 3
f = lambdify([[x1,x2,x3,x4]], func, "numpy")
gf = lambdify([[x1,x2,x3,x4]], func.diff([[x1, x2,x3,x4]]), "numpy")
grad_f = lambda x_arr : np.array(gf(x_arr)).reshape(1,4)

A = np.array([[1, 1, 1, 0],
              [1, 1, 0, 1]])
b = np.array([2,5]).reshape(2,1)
x1_bounds = [2, 5]
x2_bounds = [-1, 6]
x3_bounds = [0, 4]
x4_bounds = [0, 10]
bounds = [x1_bounds, x2_bounds, x3_bounds, x4_bounds]

### Useful Functions

In [ ]:
np_str = lambda x_k : np.array2string(x_k.reshape(len(x_k)), precision=3, separator=',')

f_str = lambda x : "{0:.4f}".format(x)

In [ ]:
class OutputTable:    
    def __init__(self):
        self.table = pd.DataFrame([],columns=['k', 'x^k', 'f(x^k)', 'd^k', 'a^k', 'x^k+1'])
    def add_row(self, k, xk, fxk, dk, ak, xkp):
        self.table.loc[len(self.table)] = [k, np_str(xk), f_str(np.asscalar(fxk)), np_str(dk), ak, np_str(xkp)]
    def print_latex(self):
        print(self.table.to_latex(index=False))

### Exact Line Search

In [ ]:
def GoldenSection(f,epsilon=0.005, a=-1000,b=1000):
    golden_ratio = (1+np.sqrt(5))/2
    gama = 1/golden_ratio
    iteration = 0
    x_1 = b - gama*(b-a)
    x_2 = a + gama*(b-a)
    fx_1 = f(x_1)
    fx_2 = f(x_2)
    while (b-a) >= epsilon:
        iteration+=1
        if(fx_1 >= fx_2):
            a = x_1
            x_1 = x_2
            x_2 = a + gama*(b-a)
            fx_1 = fx_2
            fx_2 = f(x_2)
        else:
            b = x_2
            x_2 = x_1
            x_1 = b - gama*(b-a)
            fx_2 = fx_1
            fx_1 = f(x_1)
    x_star = (a+b)/2
    fx_star = f(x_star)
    return x_star

In [ ]:
''' 
def BisectionMethod(f,epsilon, a=-100,b=100) :
    iteration=0
    while (b - a) >= epsilon:
        x_1 = (a + b) / 2
        fx_1 = f(x_1)
        if f(x_1 + epsilon) <= fx_1:
            a = x_1
        else:
            b = x_1
        iteration+=1
    x_star = (a+b)/2
    return x_star
'''
def ExactLineSearch(f, x0, d, eps=0.0000000001):
    alpha = Symbol('alpha')
    function_alpha = f(np.array(x0)+alpha*np.array(d))
    f_alp = lambdify(alpha, function_alpha, 'numpy')
    alp_star = GoldenSection(f_alp, epsilon=eps)
    return alp_star

## Reduced Gradient Method

In [ ]:
def determineBasicAndNonbasics(xk, bounds, m):
    basics = []
    nonbasics = []
    for i in range(len(bounds)):
        if(boundsbounds[i][0] < xk[i] < bounds[i][1]):
            basics.append(i)
        else:
            nonbasics.append(i)
    return basics, nonbasics

In [ ]:
xk = np.array([2, -1, 1, ])

[3, 1, 2, 4]

In [ ]:
def ReducedGradient(x0, f, gradf, A, b, bounds):
    return